In [1]:
import tensorflow as tf
from tensorflow import keras as kr
from keras import layers, Sequential
import numpy as np
import pandas as pd
import os
from env_paths import *

In [18]:
INPUT_COLS = ["t_ext", "h_ext"]
TARGET_COLS = ["t_C_cal","h_C_cal","t_L_cal","h_L_cal"]
MIN_MAX_COLS_VALUES = {"t_ext": (-50, 50), "h_ext": (0, 100), "t_C_cal": (-50, 50), 
                       "t_L_cal": (-50, 50), "h_C_cal": (0, 100), "h_L_cal": (0, 100)}

In [16]:
def train_validate_test_split(df, train_size=.6, val_size=.2, seed=None):
    np.random.seed(seed)
    per = np.random.permutation(df.index)
    size = len(df.index)
    train_end = int(train_size * size)
    validate_end = int(val_size * size) + train_end
    train = df.iloc[per[:train_end]]
    val = df.iloc[per[train_end:validate_end]]
    test = df.iloc[per[validate_end:]]
    return train, val, test

In [20]:
files = os.listdir(CLASSIFIED_SENSOR_PATH)
df = pd.read_csv(CLASSIFIED_SENSOR_PATH + files[0])
df = df.drop(["ts"], axis=1)

# Normalizacion de cada columna empleando (xi – minimo(x)) / (máximo(x) – minimo(x))
for col, (min, max) in MIN_MAX_COLS_VALUES.items():
    df[col]=(df[col]-min)/(max-min)

train_size = 0.70
val_size = 0.15
train, val, test = train_validate_test_split(df, train_size=train_size, val_size=val_size)

X_train = np.array(train[INPUT_COLS])
y_train = np.array(train[TARGET_COLS])

X_val = np.array(val[INPUT_COLS])
y_val = np.array(val[TARGET_COLS])

X_test = np.array(test[INPUT_COLS])
y_test = np.array(test[TARGET_COLS])


In [37]:
# Instanciacion del modelo, creacion de capas y declaracion de las neuronas por capas y su funcion de activacion
model = Sequential()
inputs = kr.Input(shape=(2, 1))
x = layers.GRU(100, activation="sigmoid", return_sequences=True)(inputs)
x = layers.GRU(50)(x)
# x = layers.Dense(30, activation="sigmoid")(x)
outputs = layers.Dense(4, activation="sigmoid")(x)

model = kr.Model(inputs=inputs, outputs=outputs)
model.compile(loss="mse", optimizer=kr.optimizers.Adam(learning_rate=0.05), metrics=['accuracy'])
model.summary()

# Entrenamiento de la RNA
model.fit(X_train, y_train, epochs=25, batch_size=64, verbose=1, validation_data=(X_val, y_val))


predictions = model.predict(X_test)

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 2, 1)]            0         
                                                                 
 gru_16 (GRU)                (None, 2, 100)            30900     
                                                                 
 gru_17 (GRU)                (None, 50)                22800     
                                                                 
 dense_9 (Dense)             (None, 4)                 204       
                                                                 
Total params: 53,904
Trainable params: 53,904
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
1324/1324 [==============================] - 14s 8ms/step - loss: 0.0028 - accuracy: 0.6428 - val_loss: 0.0019 - val_accuracy: 0.6717
Epoch 2/25
1324/1324 [====================

In [38]:
# Deshacemos la normalizacion de los datos predichos
predictions = pd.DataFrame(data=predictions, columns=TARGET_COLS)

for col in TARGET_COLS:
    min, max = MIN_MAX_COLS_VALUES[col]
    predictions[col] = predictions[col] * (max - min) + min

In [15]:
model.save("./ann_model", overwrite=True)

INFO:tensorflow:Assets written to: ./ann_model\assets


INFO:tensorflow:Assets written to: ./ann_model\assets
